In [6]:
from excellib import set_excel_color_daily
from stocklib import get_stock_price, get_OTC_price, get_otc, get_investor, get_ratio
from stocklib import cal_bar, cal_KD, check_10days, check_10days_otc, get_trust_today
from stocklib import convert_num
from stock_plot import plot_K_chart

from finance import get_yearly, get_df_monthly, get_df_monthly_check, get_df_quarterly
from finance import get_this_quarter, get_quarter_income, get_quarter_finance, set_growth_eps
from finance import get_df_quarterly_all, get_df_yearly_all

In [7]:
import requests
import shutil
import time
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd
import numpy as np
import yfinance as yf
import json
import matplotlib.pyplot as plt
import mplfinance as mpf
import seaborn as sns
%matplotlib inline

# ua = UserAgent()
root = '../'
path_trust = root + 'trust/'
path_check = root + 'daily_check/'

## 三大法人

In [8]:
file_track = root + 'investors_track_2.xlsx'
shutil.copy2(file_track, file_track.replace('.xlsx', '_backup.xlsx')) # complete target filename given

df_track = pd.read_excel(file_track, index_col=None, engine='openpyxl')
df_track['id'] = df_track['id'].astype(str)

lst_track_tmp = list(set(df_track['id']))
lst_track = lst_track_tmp
print('追蹤股票數：', len(lst_track), lst_track)

追蹤股票數： 1 ['1612']


In [9]:
d_today = date.today() #- timedelta(days=1)

today_tw = d_today - relativedelta(years=1911)
this_year = d_today.year
this_quarter = get_this_quarter()
n = 1
if d_today.weekday() == 0:
    n = 3
d_yesterday = d_today - timedelta(days=n)
s_yesterday = (d_today - timedelta(days=n)).strftime("%Y%m%d")
s_today_md = (d_today).strftime("%m/%d")
s_yesterday_md = (d_today - timedelta(days=n)).strftime("%m/%d")
s_today = d_today.strftime("%Y%m%d")

s_today_tw = today_tw.strftime("%Y/%m/%d").lstrip('0')
s_thismonth_tw = today_tw.strftime("%Y/%m").lstrip('0')
print('今天:', d_today)
# 台灣證券交易所

今天: 2022-11-14


In [10]:
df_today = get_stock_price(d_today)
df_today.head(2)

取得今日上市股票資訊: 20221114


,name,amount,open,high,low,close
id,,,,,,
0050,元大台灣50,18631,109.80,110.4,109.40,110.0
0051,元大中型100,42,53.35,54.0,53.35,54.0


# 取得交易價量前30名

In [11]:
df_amt = df_today.copy()
df_amt['value_amt'] = df_amt['amount'] * df_amt['close'] # 成交量 * 收盤價(億)
df_amt['value_amt'] //= 100000
df_amt.sort_values('value_amt', ascending=False, inplace=True)
lst_amt = list(df_amt.head(30).index)
print('today:', lst_amt)

path_amt30 = './amt30.txt'
with open(path_amt30, 'r') as f:
    lst_amt30_yesterday = f.read().split('\n')
print('yesterday:', lst_amt30_yesterday)

with open(path_amt30, 'w') as f:
    for stock_id in lst_amt:
        f.write(stock_id + "\n")
lst_amt30_new = [i for i in lst_amt if i not in lst_amt30_yesterday]
print('new:', lst_amt30_new)
df_amt = df_amt.head(30)
df_amt_new = df_amt.reset_index()
df_amt_new[df_amt_new['id'].isin(lst_amt30_new)]

today: ['2330', '8046', '3037', '2603', '2317', '2454', '1605', '2303', '3035', '1795', '6533', '3661', '3443', '5871', '3189', '2609', '00637L', '8996', '00632R', '2412', '3034', '1101', '3454', '6669', '6415', '0050', '2379', '2327', '2308', '3406']
yesterday: ['2330', '3661', '2317', '2603', '3037', '8046', '3454', '3443', '6533', '2609', '2498', '8996', '6669', '00637L', '2327', '6781', '2618', '1477', '6153', '3406', '2615', '2376', '2412', '2303', '5258', '8478', '4919', '2308', '4968', '3046', '']
new: ['2454', '1605', '3035', '1795', '5871', '3189', '00632R', '3034', '1101', '6415', '0050', '2379']


,id,name,amount,open,high,low,close,value_amt
5,2454,聯發科,9849,705.00,710.00,694.00,697.00,68.0
6,1605,華新,141124,42.75,44.80,42.35,44.55,62.0
8,3035,智原,32420,159.00,165.00,157.00,163.50,53.0
9,1795,美時,25385,173.00,174.50,166.00,171.00,43.0
13,5871,中租-KY,16671,176.00,190.00,174.50,190.00,31.0
14,3189,景碩,25605,120.00,122.50,118.50,119.00,30.0
18,00632R,元大台灣50反1,388171,6.01,6.03,5.97,5.99,23.0
20,3034,聯詠,7668,301.00,301.50,293.50,298.00,22.0
21,1101,台泥,62302,32.45,34.40,32.25,34.15,21.0
24,6415,矽力*-KY,4313,450.50,478.00,446.00,468.00,20.0


In [12]:
df_yesterday = get_stock_price(d_yesterday)
df_yesterday.head(2)

取得今日上市股票資訊: 20221111


,name,amount,open,high,low,close
id,,,,,,
0050,元大台灣50,35134,108.55,109.00,107.8,109.0
0051,元大中型100,47,53.00,53.15,52.8,53.0


In [13]:
df_investor = get_investor(d_today)
df_investor.head(2)

外資, 投信同步買超筆數： 12479


,investor,trust,dealer_T,dealer_F
id,,,,
2303,58858,1586,-1197,-2830
2002,34829,-455,-2440,251


In [14]:
lst_investor1 = list(df_investor[  (df_investor['trust']>0)].index) # (df_investor['investor']>=0) &
lst_investor2 = list(df_investor[ (df_investor['investor']>1000) & (df_investor['trust']==0)].index)
lst_investor = set(lst_investor1 + lst_investor2)
print(len(lst_investor))
print(list(lst_investor))

148
['00633L', '4927', '2014', '4961', '1524', '2368', '2344', '1590', '2104', '00632R', '4906', '5608', '6235', '0050', '9910', '00881', '1710', '00753L', '00892', '2881', '2637', '1303', '2882', '00830', '3008', '2633', '1301', '2615', '2330', '3045', '6770', '2897', '2352', '2614', '2801', '2888', '1907', '6781', '1308', '6719', '1313', '2303', '1477', '2454', '3189', '00676R', '4977', '00891', '6269', '1802', '2845', '2618', '8046', '3037', '1304', '5880', '1476', '2885', '1326', '2331', '2353', '1444', '1440', '5876', '5225', '1909', '080713', '6491', '8028', '3062', '2515', '2105', '3059', '1216', '2889', '1309', '2731', '6515', '1101', '1708', '00671R', '2405', '2886', '5607', '1609', '5284', '2328', '6116', '6282', '2880', '2504', '2323', '2542', '5269', '00655L', '2027', '2023', '00637L', '4968', '2887', '3706', '2337', '2723', '4915', '6505', '2601', '2031', '4919', '2809', '1314', '00631L', '2867', '00900', '00893', '1409', '2474', '00664R', '2327', '2603', '9904', '2891', '

In [15]:
# df_otc_trust = get_otc(d_today)
# df_otc_trust.set_index('代號', inplace=True)
# df_otc_trust['check'] = ''
# df_otc_trust.head(2)

In [16]:
# df_ratio = get_ratio(d_today) # 殖利率、本益比、股價淨值比
# df_ratio_today = df_ratio.copy()
# df_ratio_today = df_ratio_today[df_ratio_today['PER']!='-']
# df_ratio_today['PER'] = df_ratio_today['PER'].astype(float)
# # 殖利率 > 5%、本益比PER<10、股價淨值比 PBR <1
# df_ratio_today = df_ratio_today[(df_ratio_today['dividend%']>5) & (df_ratio_today['PER']<10) & (df_ratio_today['PBR']<1)]
# df_ratio_today.head(2)

In [17]:
# df_cheap_today = df_today[df_today.index.isin(df_ratio_today.index)]

# df_cheap = pd.concat([df_cheap_today, df_ratio_today], axis=1, join="inner")
# df_cheap = df_cheap[df_cheap['amount']>2000]
# df_cheap.reset_index(inplace=True)
# df_cheap['start'] = s_yesterday_md#today.strftime("%m/%d")

# file_cheap = root + 'cheap.xlsx'
# shutil.copy2(file_cheap, file_cheap.replace('.xlsx', '_backup.xlsx')) # complete target filename given
# df_cheap_old = pd.read_excel(file_cheap, index_col=None, engine='openpyxl')
# df_cheap_old['id'] = df_cheap_old['id'].astype(str)

# df_cheap_new = pd.concat([df_cheap_old, df_cheap])
# df_cheap_new.sort_values('dividend%', ascending=False)

# set_excel_color_daily(df_cheap_new, file_cheap)

In [18]:
## 今日符合篩選條件，新增至追蹤
# 漲幅 > 1.5%，K棒 > 1%
df_add = df_today.loc[lst_investor,:]
df_add.sort_index(inplace=True)

df_add['rise%'] = round((df_add['close'] - df_yesterday['close'])/df_yesterday['close']*100, 2)  # 漲跌%
df_add['K%'] = round( (df_add['close'] - df_add['open'])/df_add['open']*100, 2 )    # K棒%
df_add = df_add[ (df_add['rise%']>1.5) & (df_add['K%']>1) ]
df_add = check_10days(df_add)
lst_add_ori = list(df_add.index)
print('今日新增:',len(lst_add_ori),'筆\n', lst_add_ori)

lst_add = [i for i in df_add.index if i not in list(lst_track)] # 排除掉已經在track裡的股票
df_add = df_add.loc[lst_add,:]
df_add

C:\Users\Mia\AppData\Local\Temp\ipykernel_3632\3510989221.py:3: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_add = df_today.loc[lst_investor,:]


查無資料 Hist < 10 1
1101, 收盤價：34.15, 10日內最高價：32.45, 布林通道差：3.69, 布林帶寬：0.12
1308, 收盤價：26.85, 10日內最高價：26.6, 布林通道差：2.81, 布林帶寬：0.11
1309, 收盤價：20.8, 10日內最高價：20.65, 布林通道差：1.37, 布林帶寬：0.07
1313, 收盤價：13.15, 10日內最高價：12.7, 布林通道差：0.91, 布林帶寬：0.07
1409, 收盤價：17.35, 10日內最高價：17.05, 布林通道差：1.82, 布林帶寬：0.11
1524, 收盤價：23.8, 10日內最高價：23.45, 布林通道差：3.13, 布林帶寬：0.14
1609, 收盤價：18.9, 10日內最高價：17.75, 布林通道差：2.34, 布林帶寬：0.14
1710, 收盤價：17.75, 10日內最高價：17.65, 布林通道差：0.9, 布林帶寬：0.05
2014, 收盤價：22.45, 10日內最高價：21.1, 布林通道差：3.1, 布林帶寬：0.15
2023, 收盤價：15.15, 10日內最高價：14.9, 布林通道差：0.88, 布林帶寬：0.06
2027, 收盤價：40.8, 10日內最高價：40.5, 布林通道差：3.29, 布林帶寬：0.09
2104, 收盤價：20.6, 10日內最高價：20.55, 布林通道差：2.27, 布林帶寬：0.12
2201, 收盤價：49.5, 10日內最高價：46.3, 布林通道差：6.29, 布林帶寬：0.14
2355, 收盤價：30.95, 10日內最高價：29.2, 布林通道差：3.56, 布林帶寬：0.13
2382, 收盤價：73.0, 10日內最高價：70.7, 布林通道差：6.13, 布林帶寬：0.09
2542, 收盤價：43.35, 10日內最高價：42.65, 布林通道差：3.57, 布林帶寬：0.09
2723, 收盤價：110.0, 10日內最高價：106.5, 布林通道差：14.6, 布林帶寬：0.14
2897, 收盤價：8.41, 10日內最高價：8.3, 布林通道差：0.52, 布林帶寬：0.06
3062, 收盤價：23.95, 10日內最高價：23.9, 

,name,amount,open,high,low,close,rise%,K%,check
id,,,,,,,,,
1101,台泥,62302,32.45,34.40,32.25,34.15,6.22,5.24,9.0
1308,亞聚,1836,26.10,26.95,26.10,26.85,2.87,2.87,9.0
1309,台達化,1820,20.15,20.85,20.05,20.80,3.48,3.23,9.0
1313,聯成,4824,12.40,13.20,12.40,13.15,6.05,6.05,9.0
1409,新纖,5405,16.80,17.55,16.80,17.35,3.27,3.27,9.0
1524,耿鼎,12629,22.85,23.95,22.70,23.80,5.78,4.16,9.0
1609,大亞,14266,17.40,18.90,17.35,18.90,9.88,8.62,9.0
1710,東聯,1528,17.35,17.80,17.35,17.75,2.90,2.31,9.0
2014,中鴻,38834,20.70,22.45,20.70,22.45,9.78,8.45,9.0


In [19]:
print('量價前30名:', lst_amt)
df_amt = check_10days(df_amt)
lst_amt = list(df_amt.index)
print('量價前30名，10日盤整:',len(lst_amt),'筆', lst_amt)

量價前30名: ['2330', '8046', '3037', '2603', '2317', '2454', '1605', '2303', '3035', '1795', '6533', '3661', '3443', '5871', '3189', '2609', '00637L', '8996', '00632R', '2412', '3034', '1101', '3454', '6669', '6415', '0050', '2379', '2327', '2308', '3406']
1101, 收盤價：34.15, 10日內最高價：32.45, 布林通道差：3.69, 布林帶寬：0.12
量價前30名，10日盤整: 1 筆 ['1101']


# 預測EPS

In [20]:
def get_EPS(lst_stock_id, filename):
    df_yearly_all = get_df_yearly_all() # 寫入歷年EPS、本益比，估算今年股價min, max範圍
    df_quarterly_all = get_df_quarterly_all()
    df_estimate = pd.DataFrame(columns = list(df_quarterly_all.columns)+['普通股數'])

    # 取得公司發行股數
    path_company_stock_num = root + 'company_list.xlsx'
    df_company_stock_num = pd.read_excel(path_company_stock_num, engine='openpyxl', dtype={'stock_id':str})
    df_company_stock_num.set_index('stock_id', inplace=True)
    
    lst_eps = []
    for i in range(len(lst_stock_id)):
        stock_id = lst_stock_id[i]
        t_wait = np.random.randint(5,15)
        print(stock_id, ', wait ', t_wait)
        df = get_df_quarterly(stock_id)

        if df is None:
            print('df is None')
            continue
        if df.loc[df.index[0], '毛利率(%)'] < 20:
            print(stock_id, '：毛利率', df.loc[df.index[0], '毛利率(%)'], '< 20%，跳過')
            continue
        elif df.loc[df.index[0], '營益率(%)'] < 8:
            print(stock_id, '：營益率' , df.loc[df.index[0], '營益率(%)'] , ' < 8%，跳過')
            continue

        lst_eps.append(stock_id)
        # 計算歷年同季的營收成長率、EPS
        df = set_growth_eps(df, df_today, stock_id)

        # 將年度EPS寫到歷年excel裡
        lst_year = set(df.index.to_series().apply(lambda x:x[:4]))
        min_year = int(min(lst_year)) # 若最小年是2018 ，只能確定2019都有資料
        lst_year = [y for y in lst_year if int(y) > min_year]
        lst_EPS = [{'year':y, 'eps': df.loc[str(y)+'Q1', 'EPS(Y)']} for y in lst_year]

        df.reset_index(inplace=True)
        df = df.rename(columns={'index':'quarter'}) # reset_index後column名稱是index

        stock_name = ""
        if stock_id in df_today.index:
            stock_name = df_today.loc[stock_id, 'name']
            df_yearly = get_yearly(stock_id, lst_EPS, df['quarter'][0][:4], stock_name)    
            df_yearly_all = df_yearly_all.append(df_yearly, ignore_index=True)

        df.insert(0, 'name', value=stock_name)
        df.insert(0, 'stock_id', value=stock_id)

        df_quarterly_all = df_quarterly_all.append(df, ignore_index=True)
        time.sleep(t_wait)

        # ===============================
        # 預測EPS ( check_EPS )
        # ===============================
        df_new_quarter = get_quarter_finance(stock_id, df, this_year, this_quarter, df_company_stock_num, df_today)
        df_concat = pd.concat([df, df_new_quarter])
        df_concat.sort_values('quarter', ascending=False, inplace=True)
        df_estimate = df_estimate.append(df_concat)

    #預測EPS ( check_EPS )
    df_estimate.to_excel(path_check+'estimate_{}_{}.xlsx'.format(filename, s_today), index=False)
    return lst_eps

In [21]:
print('今日觀察：', len(lst_add_ori), '筆')
lst_add_ori = get_EPS(lst_add_ori, 'add')
print('EPS檢查', len(lst_add_ori), '筆\n', lst_add_ori)

今日觀察： 23 筆
1101 , wait  12


C:\Users\Mia\.virtualenvs\stock-8IVDzipY\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


1101 ：毛利率 7.4 < 20%，跳過
1308 , wait  14


C:\Users\Mia\.virtualenvs\stock-8IVDzipY\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


1308  wait  7
'2022'


...

6768  wait  6


C:\Users\Mia\AppData\Local\Temp\ipykernel_3632\298807737.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_yearly_all = df_yearly_all.append(df_yearly, ignore_index=True)
C:\Users\Mia\AppData\Local\Temp\ipykernel_3632\298807737.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_quarterly_all = df_quarterly_all.append(df, ignore_index=True)
C:\Users\Mia\AppData\Local\Temp\ipykernel_3632\298807737.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_estimate = df_estimate.append(df_concat)


9904 , wait  10


C:\Users\Mia\.virtualenvs\stock-8IVDzipY\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


9904 ：營益率 5.2  < 8%，跳過
EPS檢查 10 筆
 ['1308', '1409', '1524', '2027', '2201', '2542', '2897', '5607', '5608', '6768']


In [22]:
print('量價前30名：', lst_amt)
lst_amt = get_EPS(lst_amt, 'amt30')
print('EPS檢查：',len(lst_amt), '筆\n', lst_amt)

量價前30名： ['1101']
1101 , wait  11
1101 ：毛利率 7.4 < 20%，跳過
EPS檢查： 0 筆
 []


C:\Users\Mia\.virtualenvs\stock-8IVDzipY\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# 今日新增 && 在量價前30名

In [23]:
print('今日新增 && 在量價前30名', list(set(lst_add_ori)& set(lst_amt)) )

今日新增 && 在量價前30名 []


# 畫日K線圖、週K線圖

In [24]:
file_name = root + 'stock_20210825_1203.json'
with open(file_name) as f:
    dic_stock = json.load(f)

latest_date_file = datetime.strptime(sorted(dic_stock)[-1], '%Y-%m-%d').date()
print('目前記錄日期至:',latest_date_file, '新增前:', len(dic_stock), '筆')

目前記錄日期至: 2022-10-20 新增前: 258 筆


In [25]:
import datetime
import time
import requests
from io import StringIO
import pandas as pd
import numpy as np

def crawl_price(date):
    r = requests.post('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + str(date).split(' ')[0].replace('-','') + '&type=ALL')
    print(str(date).split(' ')[0].replace('-',''))
    ret = pd.read_csv(StringIO("\n".join([i.translate({ord(c): None for c in ' '}) 
                                        for i in r.text.split('\n') 
                                        if len(i.split('",')) == 17 and i[0] != '='])), header=0)
    ret = ret.set_index('證券代號')
    ret['成交金額'] = ret['成交金額'].str.replace(',','')
    ret['成交股數'] = ret['成交股數'].str.replace(',','')
    return ret


data_stock = {}
date_today = d_today
n_days = (d_today - latest_date_file).days
fail_count = 0
allow_continuous_fail_count = 5
print('須補天數:', n_days)
while len(data_stock) < n_days:
    print('parsing', date_today)
    # 使用 crawPrice 爬資料
    try:
        # 抓資料
        if date_today.weekday()  in [5,6]:
            print('Weekend!')
            date_today -= datetime.timedelta(days=1)
            continue
        data_stock[date_today] = crawl_price(date_today)
        if not str(date_today) in dic_stock.keys():
            dic_stock[str(date_today)] = data_stock[date_today].to_dict()
            print("add to dict:", str(date_today))
        print('success!')
        fail_count = 0
    except:
        # 假日爬不到
        print('fail! check the date is holiday')
        fail_count += 1
        if fail_count == allow_continuous_fail_count:
            raise
        break
    
    # 減一天
    date_today -= datetime.timedelta(days=1)
    time.sleep(10)

print('新增後:', len(dic_stock))
with open(file_name, 'w') as f:
    json.dump(dic_stock, f)

須補天數: 25
parsing 2022-11-14
20221114
add to dict: 2022-11-14
success!
parsing 2022-11-13
Weekend!
parsing 2022-11-12
Weekend!
parsing 2022-11-11
20221111
add to dict: 2022-11-11
success!
parsing 2022-11-10
20221110
add to dict: 2022-11-10
success!
parsing 2022-11-09
20221109
add to dict: 2022-11-09
success!
parsing 2022-11-08
20221108
add to dict: 2022-11-08
success!
parsing 2022-11-07
20221107
add to dict: 2022-11-07
success!
parsing 2022-11-06
Weekend!
parsing 2022-11-05
Weekend!
parsing 2022-11-04
20221104
add to dict: 2022-11-04
success!
parsing 2022-11-03
20221103
add to dict: 2022-11-03
success!
parsing 2022-11-02
20221102
add to dict: 2022-11-02
success!
parsing 2022-11-01
20221101
add to dict: 2022-11-01
success!
parsing 2022-10-31
20221031
add to dict: 2022-10-31
success!
parsing 2022-10-30
Weekend!
parsing 2022-10-29
Weekend!
parsing 2022-10-28
20221028
add to dict: 2022-10-28
success!
parsing 2022-10-27
20221027
add to dict: 2022-10-27
success!
parsing 2022-10-26
20221026
ad

In [26]:
def plot_daily_weekly_K(lst_stock_id, dic_stock):
    df_close = pd.DataFrame({k:d['收盤價'] for k,d in dic_stock.items()}).transpose()
    df_close.index = pd.to_datetime(df_close.index)

    df_open = pd.DataFrame({k:d['開盤價'] for k,d in dic_stock.items()}).transpose()
    df_open.index = pd.to_datetime(df_open.index)

    df_high = pd.DataFrame({k:d['最高價'] for k,d in dic_stock.items()}).transpose()
    df_high.index = pd.to_datetime(df_high.index)

    df_low = pd.DataFrame({k:d['最低價'] for k,d in dic_stock.items()}).transpose()
    df_low.index = pd.to_datetime(df_low.index)

    df_volume = pd.DataFrame({k:d['成交股數'] for k,d in dic_stock.items()}).transpose()
    df_volume.index = pd.to_datetime(df_volume.index)
    
    # =========================== 畫K線圖 =================================================
    for stock_id in lst_stock_id:
        thisyear = '2021'
        print(stock_id)
        if not stock_id in df_close.keys():
            print(stock_id, '查無資料')
            continue
    #     stock_id = '2603'
        dic_a_stock = {
            'close':df_close[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
            'open':df_open[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
            'high':df_high[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
            'low':df_low[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
            'volume': df_volume[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
        }

        # Daily DataFrame
        df_stock = pd.DataFrame(dic_a_stock)
        df_stock.insert(0,'stock_id', stock_id)
        df_stock.sort_index(inplace=True)

        fig_daily = plot_K_chart(df_stock, 'daily')

        # Weekly DataFrame
        period_type = '1W'
        df_week = df_stock.resample(period_type).last()
        df_week['close'] = df_stock['close'].resample(period_type).last()
        df_week['open'] = df_stock['open'].resample(period_type).first()
        df_week['high'] = df_stock['high'].resample(period_type).max()
        df_week['low'] = df_stock['low'].resample(period_type).min()
        df_week['volume'] = df_stock['volume'].resample(period_type).sum()

        fig_weekly = plot_K_chart(df_week, 'weekly')

In [27]:
print('今日新增 && 在量價前30名', list(set(lst_add_ori)& set(lst_amt)) )

今日新增 && 在量價前30名 []


In [28]:
print('今日觀察：', len(lst_add_ori), '筆')
plot_daily_weekly_K(lst_add_ori, dic_stock)

今日觀察： 10 筆
1308


ModuleNotFoundError: No module named 'talib'

In [ ]:
print('量價前30名：', len(lst_amt),'筆\n', lst_amt)
lst_amt_without_add = [i for i in lst_amt if i not in lst_add_ori]
plot_daily_weekly_K(lst_amt_without_add, dic_stock)

In [ ]:
# df_yesterday = df_yesterday[df_yesterday.index.isin(df_add.index)]
# df_yesterday['rise%'] = round((df_yesterday['close'] - df_yesterday['open'])/df_yesterday['close']*100, 2) # 漲跌%
# df_yesterday['K%'] = round( (df_yesterday['close'] - df_yesterday['open'])/df_yesterday['open']*100, 2 )    # K棒%
# df_yesterday['check']=0
# df_yesterday.insert(1, 'date', s_yesterday_md)
# df_yesterday.reset_index(inplace=True)
# df_yesterday

In [ ]:
# df_ori = df_today.loc[lst_track,:]
# df_ori['rise%'] = round((df_ori['close'] - df_ori['open'])/df_ori['close']*100, 2) # 漲跌%
# df_ori['K%'] = round( (df_ori['close'] - df_ori['open'])/df_ori['open']*100, 2 )    # K棒%

# df_today = pd.concat([df_ori, df_add])
# print('ori track:',lst_track, ', add：', df_add.index)
# df_today.head(2)

In [ ]:
# def get_stoploss(stock_id):
#     series = df_today.loc[stock_id,:]
#     # stock_id = series['id']
#     if  series['K%'] > 6:
#         stop_point = round(series['open'] + (series['close'] - series['open'])/2, 1)
#     elif series['K%'] > 2:
#         stop_point = series['open']
#     elif stock_id in df_track['id'].unique(): # stock_id存在於原本追蹤清單中
#         stop_point = df_track[df_track['id']==stock_id]['stoploss'].iloc[0]
#     else:
#         stop_point = series['open'] # stock_id不存在於原本追蹤清單中
#     return stop_point

# def cal_mean_51018(stock_id):
#     time.sleep(0.8)
#     data_yf = yf.Ticker(stock_id+'.TW')
#     df = data_yf.history(period="1mo")
#     df.sort_index(ascending=False, inplace=True)
    
#     price_5 = round(np.mean(df['Close'][:5]),2)
#     price_10 = round(np.mean(df['Close'][:10]),2)
#     price_20 = round(np.mean(df['Close'][:20]),2)
    
#     prev_5 = round(np.mean(df['Close'][1:6]),2)
#     prev_10 = round(np.mean(df['Close'][1:11]),2)
#     prev_20 = round(np.mean(df['Close'][1:21]),2)
    
#     slope_5 = round((price_5 - prev_5)*100/price_5, 2)
#     slope_10 = round((price_10 - prev_10)*100/price_10, 2)
#     slope_20 = round((price_20 - prev_20)*100/price_20, 2) # 除上當日均價 標準化
    
#     k, d = cal_KD(stock_id, df, df_track, df_today)
#     return pd.Series([price_5, price_10, price_20, slope_5, slope_10, slope_20, k, d])

# def get_start_date(stock_id):
#     df_ori = df_track[df_track['id']==stock_id]
#     if df_ori.empty:  
#         return d_today.strftime("%m/%d")
#     else:
#         return df_ori['start'].iloc[0]

In [ ]:
# # 停損點、上影線、下影線
# df_today.insert(1, 'date', s_today_md)
# df_today['stoploss'] = df_today.index.to_series().apply(get_stoploss)
# df_today[ ['up_bar', 'low_bar'] ] = df_today.apply(lambda x: cal_bar(x.open, x.high, x.low, x.close), axis=1)

# # 5, 10 ,20日均線，K,D值
# df_today[ ['close5', 'close10', 'close20', 'slope5','slope10', 'slope20', 'Kvalue', 'Dvalue'] ]  = \
#         df_today.index.to_series().apply(cal_mean_51018) # 太長了

# df_today.head(2)

In [ ]:
# df = pd.concat([df_today, df_investor, df_ratio], axis=1, join="inner")
# df.reset_index(inplace=True)

# df_all = pd.concat([df, df_yesterday])

# df_all['start'] = df_all['id'].apply(get_start_date)
# df_all.head(2)

In [ ]:
# df_new = pd.concat([df_track, df_all])
# df_new.sort_values(['id', 'date'], ascending=[True, False], inplace=True)
# df_new.head()

In [ ]:
# set_excel_color_daily(df_new, file_track)

In [ ]:
# # 目前持有股票
# file_asset = root + 'asset.xlsx'
# shutil.copy2(file_asset, file_asset.replace('.xlsx', '_backup.xlsx')) # complete target filename given

# txt = root + 'asset.txt'
# with open(txt, 'r', encoding='utf8') as f:
#     dic = json.loads(f.read())
# lst_asset = list(dic.keys())
# print(lst_asset)

# df_asset = df_new[df_new['id'].isin(lst_asset)]
# set_excel_color_daily(df_asset, file_asset)
# df_asset

# 投信買超

In [ ]:
# # 投信買超 上櫃公司
# df_otc_add = df_otc_trust.copy()
# df_otc_add = check_10days_otc(df_otc_add)
# df_otc_add.insert(0, 'date', s_today_md)
# df_otc_add.reset_index(inplace=True)

# path_otc = path_trust + 'otc.xlsx'
# shutil.copy2(path_otc, path_otc.replace('.xlsx', '_backup.xlsx'))
# df_otc_add.to_excel(path_otc, index=False)
# df_otc_add

In [ ]:
# # 當日投信買超 上市、上櫃公司
# url_trust = 'https://www.twse.com.tw/fund/TWT44U?response=html&date={}'.format(s_today)
# tables = pd.read_html(url_trust)
# df_trust = tables[0]
# df_trust = df_trust.iloc[:,[1,5]]
# df_trust.columns = ['stock_id', '買賣超']
# df_trust = df_trust.astype({'stock_id':str, '買賣超':int})
# df_trust.iloc[:,-1] = df_trust.iloc[:,-1].apply(lambda x: round(x/1000,0))
# df_trust = df_trust[df_trust['買賣超']>20]
# df_trust.set_index('stock_id', inplace=True)
# df_trust.head(2)

In [ ]:
# # 投信持股比例
# path_trust = 'D:/Mia/Stock/trust/'
# path_html = path_trust + 'StockList.html'
# tables = pd.read_html(path_html)
# df_trust_today = tables[0]
# df_trust_today.columns = df_trust_today.iloc[0,:]
# df_trust_today = df_trust_today.iloc[1:,:]
# df_trust_today.dropna(inplace=True)
# df_trust_today['今年買賣超佔發行張數'] = df_trust_today['今年買賣超佔發行張數'].apply(convert_num)
# df_trust_today['一個月買賣超佔發行張數'] = df_trust_today['一個月買賣超佔發行張數'].apply(convert_num)
# df_trust_today['三個月買賣超佔發行張數'] = df_trust_today['三個月買賣超佔發行張數'].apply(convert_num)
# df_trust_today = df_trust_today[(df_trust_today['今年買賣超佔發行張數']>2) & \
#                                 (df_trust_today['三個月買賣超佔發行張數']>0) & (df_trust_today['一個月買賣超佔發行張數']>0)]
# lst_trust_today = [str(i) for i in list(df_trust_today['代號'])]
# print(lst_trust_today)

# # 存檔
# path_trust_list = path_trust + 'trust.xlsx'
# shutil.copy2(path_trust_list, path_trust_list.replace('.xlsx', '_backup.xlsx'))

# df_trust_list = pd.read_excel(path_trust_list, engine='openpyxl')
# print('old：', df_trust_list.shape)
# print('new：', df_trust_today.shape)
# df_trust_new = pd.concat([df_trust_list,df_trust_today]).drop_duplicates().reset_index(drop=True)
# print('concat：', df_trust_new.shape)
# df_trust_new.sort_values(['代號', '更新日期'], ascending=[True, False])
# df_trust_new.to_excel(path_trust_list, index=False)
# df_trust_new.head(1)

In [ ]:
# lst_trust_add = [i for i in lst_trust_today if i in df_today.index]
# print('投信買超：', len(lst_trust_today), lst_trust_today)
# print('上市公司：', len(lst_trust_add), lst_trust_add)

# df_trusk_add = df_today.loc[lst_trust_add,:]
# df_trusk_add.sort_index(inplace=True)

# # df_trusk_add['rise%'] = round((df_trusk_add['close'] - df_yesterday['close'])/df_yesterday['close']*100, 2)  # 漲跌%
# df_trusk_add['K%'] = round( (df_trusk_add['close'] - df_trusk_add['open'])/df_trusk_add['open']*100, 2 )    # K棒%
# df_trusk_add = df_trusk_add[df_trusk_add['K%']>1] # df_trusk_add['rise%']>1.5) & 
# df_trusk_add = check_10days(df_trusk_add)
# df_trusk_add

In [ ]:
# df_fund = pd.read_excel(root + 'trust/fund_list.xlsx', engine='openpyxl')
# lst_fund = list(df_fund['基金編號'])
# print(lst_fund)

# path_fund_holding = path_trust+'holding.xlsx'
# shutil.copy2(path_fund_holding, path_fund_holding.replace('.xlsx', '_backup.xlsx'))
# df_holding_ori = pd.read_excel(path_trust+'fund_holding.xlsx', engine='openpyxl')
# for fund_id in lst_fund:
# #     fund_id = 'AC0053'
#     url = 'https://finet.landbank.com.tw/w/wr/wr04.djhtm?a={}'.format(fund_id)
#     tables = pd.read_html(url)
#     df_holding = tables[1]

#     col_idx = df_holding[df_holding[0]=='股票名稱'].index[0] # 取得col name index
#     df_holding.columns = df_holding.iloc[col_idx,:]
#     df_holding = df_holding.iloc[col_idx+1:-1,:]

#     df_1 = df_holding.iloc[:,:4]
#     df_2 = df_holding.iloc[:, 4:]
#     df_holding = pd.concat([df_1, df_2], ignore_index=True)
#     df_holding.dropna(inplace=True)
#     df_holding['增減'] = df_holding['增減'].apply(lambda x : x.replace('%', ''))
#     df_holding = df_holding.astype({'比例':float, '增減':float})
# #     df_holding = df_holding[df_holding['增減']>0.08]
#     df_holding.insert(0, 'date', s_today_md)
#     df_holding.insert(0, 'stock_id', fund_id)
#     df_holding_ori = df_holding_ori.append(df_holding)
#     time.sleep(0.3)

# # df_holding_new = pd.concat([df_holding_ori,df_holding])
# df_holding_ori.to_excel(path_trust+'fund_holding.xlsx', index=False)


In [ ]:
# for idx, row in df_holding_ori.iterrows():
#     df_holding_ori.at[idx, 'count'] = df_count.loc[row['股票名稱'], 'count']
# df_holding_ori.to_excel(path_trust+'fund_holding.xlsx', index=False)
# df_holding_ori.head(2)

In [ ]:

# df_trust_5days = pd.read_excel(path_trust + 'trust_5days.xlsx', engine='openpyxl')
# df_trust_5days = df_trust_5days[df_trust_5days['今年買賣超佔發行張數']>2]
# df_trust_5days = df_trust_5days[df_trust_5days['三個月買賣超佔發行張數']>0]
# df_trust_5days = df_trust_5days[df_trust_5days['一個月買賣超佔發行張數']>0]

# df_fund_holding = pd.read_excel(path_trust + 'fund_holding.xlsx', engine='openpyxl')
# lst_fund_holding = list(df_fund_holding['股票名稱'].unique())
# print(lst_fund_holding[:5])

# df_trust_5days = df_trust_5days[df_trust_5days['名稱'].isin(lst_fund_holding)]
# df_trust_5days = df_trust_5days.iloc[:,1:]
# df_trust_5days.to_excel(path_trust + 'trust_check.xlsx', index=False)
# print(list(df_trust_5days['代號']))
# df_trust_5days

# 盤中檢查

In [ ]:
# # c: 代號
# # o: 開盤價
# # z: 當盤成交價，有時候會沒有
# # v: 成交量
# # y: 昨日收盤
# import requests
# url = "https://mis.twse.com.tw/stock/api/getStockInfo.jsp?ex_ch=tse_2330.tw|tse_3008.tw"
# res = requests.get(url)
# res.json()['msgArray']

# df = pd.DataFrame(res.json()['msgArray'])
# df = df[['c','o', 'z', 'v', 'y']]
# df.columns = ['stock_id', 'open', 'price', 'volume', 'yesterday']
# df[df.columns[1:]] = df[df.columns[1:]].astype(float)
# # df = df[df['price']>df['yesterday']] # 今天漲
# df['rise%'] = round((df['price'] - df['yesterday'])/df['yesterday']*100, 2)
# df = df[df['rise%']>2] # 漲幅>2%
# df

In [ ]:
# data = yf.Ticker('2603'+'.TW')
# df_hist = data.history(period="1mo")
# df_hist.sort_index(ascending=False, inplace=True)
# df_hist.head(2)

In [ ]:
# import io
# io_buf = io.BytesIO()
# a.savefig(io_buf, format='raw')
# io_buf.seek(0)
# img_a = np.reshape(np.frombuffer(io_buf.getvalue(), dtype=np.uint8),
#                      newshape=(int(a.bbox.bounds[3]), int(a.bbox.bounds[2]), -1))

# b.savefig(io_buf, format='raw')
# io_buf.seek(0)
# img_b = np.reshape(np.frombuffer(io_buf.getvalue(), dtype=np.uint8),
#                      newshape=(int(b.bbox.bounds[3]), int(b.bbox.bounds[2]), -1))
# io_buf.close()

In [ ]:
# # img_a = np.fromstring(a.canvas.tostring_rgb(), dtype=np.uint8, sep='')
# # print(a.canvas.get_width_height()[0] * a.canvas.get_width_height()[-1] * 3)
# # print(img_a.shape)
# # img_a = img_a.reshape(a.canvas.get_width_height()[::-1] + (3,))

# figures = {'a':img_a, 'b':img_b}
# nrows, ncols = 2, 1
# fig, axeslist = plt.subplots(ncols=ncols, nrows=nrows)
# for ind,title in enumerate(figures):
#     print(ind, title)
#     axeslist.ravel()[ind].imshow(figures[title])
# #     axeslist.ravel()[ind].set_title(title)
# #     axeslist.ravel()[ind].set_axis_off()
# plt.tight_layout() # optional
# # ValueError: cannot reshape array of size 3378075 into shape (576,1728,3)

In [ ]:
# df_week = pd.DataFrame(columns=df_stock.columns)
# df_week

In [ ]:
# from talib import abstract

# def talib2df(talib_output):
#     if type(talib_output) == list:
#         ret = pd.DataFrame(talib_output).transpose()
#     else:
#         ret = pd.Series(talib_output)
#     ret.index = tsmc['close'].index
#     return ret;

# talib2df(abstract.STOCH(tsmc)).plot()
# tsmc['close'].plot(secondary_y=True)

In [ ]:
# import matplotlib.pyplot as plt
# import mplfinance as mpf
# %matplotlib inline
# import seaborn as sns

# # mplfinance內建的漲/跌標記顏色是美國的版本(綠漲紅跌)，先用mplfinance中自訂圖表外觀功能mpf.make_marketcolors()將漲/跌顏色改為台灣版本(紅漲綠跌)，
# # 接著再將這個設定以mpf.make_mpf_style()功能保存為自訂的外觀。
# mc = mpf.make_marketcolors(up='r', down='g', inherit=True)
# s  = mpf.make_mpf_style(base_mpf_style='yahoo', marketcolors=mc)

# # df_stock.index = df_stock.index.format(formatter=lambda x: x.strftime('%Y-%m-%d')) 
# df_stock.index.name = 'Date'
# fig = plt.figure(figsize=(24, 8))

# # add_axes( x初始座標, y初始座標, 寬, 高 )
# ax1 = fig.add_axes([0, 0.3, 1, 0.5])
# ax2 = fig.add_axes([0, 0, 1, 0.3])
# # ax1 = fig.add_subplot(1, 1, 1)
# # ax2 = fig.add_subplot(2, 1, 2, sharex=ax1)
# ax2.set_xticks(range(0, len(df_stock.index), 5))
# ax2.set_xticklabels(df_stock.index[::5])
# # mpf.candlestick2_ochl(ax, df_stock['open'], df_stock['close'], df_stock['high'],
# #                       df_stock['low'], width=0.6, colorup='r', colordown='g', alpha=0.75); 

# mpf.plot(df_stock, type = 'candle', mav=(10,30), ax=ax1, style=s, volume=ax2) # 必須指定volume一個軸



# 月報

#### 彙總報表 (單位：千元)
https://mops.twse.com.tw/mops/web/t21sc04_ifrs